# Notes

Exam contains 6 problems, Most of them are of intermediate complexity and follow the material from class or graded assignments. Note, that no loops are allowed in this exam, and all the solutions containing loops will be graded as 0.

For this exam you'll need [Titanic](https://www.kaggle.com/c/titanic) and [road accidents](https://www.kaggle.com/daveianhickey/2000-16-traffic-flow-england-scotland-wales) datasets.

In [1]:
%pylab inline
plt.style.use("bmh")

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
plt.rcParams["figure.figsize"] = (6,6)


In [3]:
import numpy as np
import pandas as pd
import torch

In [4]:
STUDENT = "Amir Alikulov"
ASSIGNMENT = "exam"
TEST = False

In [5]:
if TEST:
    import solutions
    total_grade = 0
    MAX_POINTS = 10

# NumPy

### 1. Filtering array (2 points).

Clip array values according to the following:

- given a two-dimensional array `arr` and threshold value `max_val`,
- find those rows, for which row values sum is `> max_val`,
- and replace largest value for each of those rows with `v` $\rightarrow$ `v - <row sum> + max_val`.

For example, consider the following array and threshold `max_val=8`:

In [6]:
a = np.array([[1, 5, 4], [-3, 2, 8]])



Row sums are:

In [7]:
a.sum(axis=1)

array([10,  7])

Since row sum for row `0` is `> max_val`, largest value in that row (`a[0, 1]`, which is `5`), must be replaced with: `5 - 10 + 8 = 3`, resulting in:

In [8]:
a_clipped = np.array([[1, 3, 4], [-3, 2, 8]])
a_clipped

array([[ 1,  3,  4],
       [-3,  2,  8]])

#### Notes:

- **do not change original array**,
- in this problem you may need to use **boolean and fancy indexing**, as well as `arr.argmax(...)`,
- you **cannot use loops**,
- input array is of **any two-dimensional shape** (including `(N,1)` and `(1,N)`), filled with **random integers**,
- there may be no rows, which satisfy threshold condition, and in that case resulting array must be identical to input array.

In [9]:
def clip_array(arr, max_val):
    """Clip array based on `max_val`."""
    arr_copy = arr.copy()
    row_sums = arr.sum(axis=1)
    rows_mask = row_sums > max_val
    max_indices = arr.argmax(axis=1)
    arr_copy[rows_mask, max_indices[rows_mask]] = arr[rows_mask, max_indices[rows_mask]] - row_sums[rows_mask] + max_val

    return arr_copy

In [10]:
PROBLEM_ID = 1

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, clip_array)

### 2. Calculate area (1 point).

In this problem you will construct a naive Monte-Carlo simulator. Provided with a 2D bounding box, you must calculate it's area:

- a bounding box is specified by maximum and minimum `x` and `y`, i.e. a bounding box is a **rectangle** between `minx` and `maxx` over `x`-axis and between `miny` and `maxy` over `y`-axis,
- all of `minx`, `maxx`, `miny`, `maxy` are `>=0` and `<=1`,
- you can sample **at most** `n_samples` points on 2D place,
- ratio of number of points inside a bounding box to total number of points is an **estimate of bounding box area**,
- estimate is considered valid, if it's **no more than 10% off of actual area value**,
- `n_samples` is chosen in such a way, that **10% error is achievable nearly always**, i.e. chances of getting more then 10% error with correct computation are negligibly small.

For example, a bounding box is `minx=0.25`, `maxx=0.5`, `miny=0.1`, `maxy=0.6`. Actual area is `0.125`. Suppose, that we sample `10000` points in unit square $x \in [0, 1],\,y \in [0, 1]$ and 1215 of them are inside the bounding box. Then, an estimate for the bounding box area is `0.1215` (with error of about 2.8%). Image below illustrates this example.

![Monte-Carlo integration example](mc.png)

In [39]:

def calc_area(minx, maxx, miny, maxy, n_samples):
    """Calculate area of bounding box."""

    x = np.random.uniform(0, 1, n_samples)
    y = np.random.uniform(0, 1, n_samples)

    mask = (x >= minx) & (x <= maxx) & (y >= miny) & (y <= maxy)
    return mask.sum() / n_samples

0.126


In [21]:
PROBLEM_ID = 2

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, calc_area)

### 3. Find outliers (3 points).

Given an array of shape `(N,2)`, filter all the rows, which are more than `thr` away from other rows. Distance metrics is Euclidean, i.e. distance between rows `i` and `j` is (in pseudocode):

```
distance(i, j) = sqrt(square(arr[i, 0] - arr[j, 0]) + square(arr[i, 1] - arr[j, 1]))
```

Distance of row `i` from other rows is:

```
distance(i) = mean(distance(i, j)), j!=i
```

Rows, which have `distance(i) > thr` must be filtered. In this problem you **cannot use loops**. Instead, use broadcasting (recall recurrence matrix problem in GA-2 and extend it to two-dimensional case).

As an example, consider 1000 samples from standard normal distribution for `x` (axis 1) and `y` (axis 0) and threshold of 2:

![Outliers filtering](outliers.png)

In [ ]:
def find_outliers(arr, thr):
    """Find outliers."""
    a = arr[:, np.newaxis, :]
    b = arr[np.newaxis, :, :]
    diff = a - b
    distances = np.sqrt(np.sum(diff**2, axis=2))
    mean_distances = np.sum(distances, axis=1) / (len(arr) - 1)
    return arr[mean_distances <= thr]


In [ ]:
PROBLEM_ID = 3

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, find_outliers)

# PyTorch

### 4. SImple derivative (1 point).

Given some value of `x0`, calculate a derivative of sigmoid function at that point. Input is a single floating point value. Output must also be a single floating point value (not a tensor!) equal to derivative of $\sigma(x)$ at `x0`.

Do not use the exact formula for the derivative, but use PyTorch `.backward()`.

In [11]:
def d_sigmoid(x0):
    """Derivative of sigmoid."""
    x = torch.tensor(float(x0), requires_grad=True)
    y = torch.sigmoid(x)
    y.backward()
    return float(x.grad)

In [12]:
PROBLEM_ID = 4

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, d_sigmoid)

# Pandas

### 5. Ratio of males travelling alone per class (1 point).

Given the Titanic dataset, calculate ratio of males travelling alove (`SibSp==0` and `Parch==0`) per class. In other words, calculate number of males travelling alone in each class, divided by number of passengers in that class.

Input is indexed with `PassengerId` and is a concatenation of train and test sets. Output must be a series, indexed by class, containing the requested ratios.

In [ ]:
def lone_males(df):
    """Calculate ratio of males travelling alone per class."""

    df_alone_males = df.loc[(df.SibSp == 0) & (df.Parch == 0) & (df.Sex == 'male')]
    return df_alone_males.groupby('Pclass').size() / df.groupby('Pclass').size()



In [ ]:
PROBLEM_ID = 5

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, lone_males)

### 6. Worst days on UK roads in 2005 (2 points).

Calculate Top-5 days with the largest number of severe accidents (`Accident_Severity < 3`).

Input is a **dataframe**, containing all the accidents in 2005 and the following columns: `date_time` (constructed in the same way, as in optional time series notebook) and `Accident_Severity`. Index is a default integer index. Result must be a list (or tuple) of dates (as a `pd.Timestamp`) with 5 elements.

In [ ]:
def worst_days(df):
    """Calculate Top 5 most severe days."""
    df['date_time'] = pd.to_datetime(df['date_time'])
    df['date'] = df['date_time'].dt.date
    top_5_days = df[df['Accident_Severity'] < 3].groupby('date').size().nlargest(5)

    return [pd.Timestamp(d) for d in top_5_days.index]


In [ ]:
PROBLEM_ID = 6

if TEST:
    total_grade += solutions.check(STUDENT, PROBLEM_ID, worst_days)

In [ ]:
if TEST:
    print(f"{STUDENT}: {int(100 * total_grade / MAX_POINTS)}")